In [134]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
from time import sleep
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Considerar a idade da empresa vs o historico de dividas

In [135]:
def get_details(numero):
    URL = "https://api-v2.idwall.co/relatorios"
    authorization = "b3818f92-5807-4acf-ade8-78a1f6d7996b"
    url_details = URL + "/{}".format(numero) + "/dados"
    while True:
        dets = requests.get(url_details, headers={"authorization": authorization})
        djson = dets.json()
        sleep(1)
        if djson['result']['status'] == "CONCLUIDO":
            break

    return dets.json()

In [136]:
df = pd.read_excel("../tabelas/base_idwall_201904.xlsx")

In [137]:
df["flag_cnpj"] = df.apply(lambda x : int(len(x["cnpj_cpf"])==14), axis=1)

In [138]:
df = df[df['flag_cnpj']==1]

In [139]:
df.rename(columns={"cnpj_cpf" : "cnpj"}, inplace=True)

In [140]:
df.head()

,id,cnpj,data_ref,data_inclusao,data_ocorrencia,natureza,valor,tipo,numero_consulta,situacao_cadastral_cnpj,sancoes_onu,pep,mun_fronteirico,data_consulta,flag_cnpj
4999,6267,19008188000142,2018-12-03,2015-06-05 00:00:00,2015-01-31 00:00:00,CÓDIGO DE ANOTAÇÃO NÃO ENCONTRADO,25397.60,pendencias,ec23b694-37ea-4cbe-85b4-cfed79f90ae6,NaN,NaN,NaN,NaN,2018-12-03,1
5000,6268,19008188000142,2018-12-03,2017-07-04 00:00:00,2016-12-21 00:00:00,NaN,1.00,cheques,ec23b694-37ea-4cbe-85b4-cfed79f90ae6,NaN,NaN,NaN,NaN,2018-12-03,1
5001,6269,19008188000142,2018-12-03,2018-06-18 00:00:00,2018-05-21 00:00:00,Falta Pagamento,892.53,protestos,ec23b694-37ea-4cbe-85b4-cfed79f90ae6,NaN,NaN,NaN,NaN,2018-12-03,1
5002,6270,19008188000142,2018-12-03,2018-04-11 00:00:00,2018-03-22 00:00:00,Falta Pagamento,560.00,protestos,ec23b694-37ea-4cbe-85b4-cfed79f90ae6,NaN,NaN,NaN,NaN,2018-12-03,1
5003,6271,19008188000142,2018-12-03,2018-04-09 00:00:00,2018-03-15 00:00:00,Falta Pagamento,560.80,protestos,ec23b694-37ea-4cbe-85b4-cfed79f90ae6,NaN,NaN,NaN,NaN,2018-12-03,1


In [30]:
js = get_details('ec23b694-37ea-4cbe-85b4-cfed79f90ae6')

In [34]:
js.get('result').get('cnpj').get("data_abertura")

'04-10-2013'

In [38]:
resp = list()
for el in df["cnpj"].unique().tolist():
    numero = df[df['cnpj']==el]["numero_consulta"].iloc[0]
    js = get_details(numero)
    data_abertura = js.get("result").get('cnpj').get('data_abertura')
    resp.append(pd.DataFrame({'cnpj' : [el], "data_abertura" : [data_abertura]}))

In [40]:
df_abertura = pd.concat(resp)

In [42]:
df_abertura["data_abertura"].iloc[0]

'04-10-2013'

In [47]:
df_abertura["data_abertura"] = df_abertura.apply(lambda x : x["data_abertura"].replace("/", "-") if not x["data_abertura"] is None else "26-04-2019", axis=1)

In [48]:
df_abertura["data_abertura"] = df_abertura.apply(lambda x : datetime.strptime(x["data_abertura"], "%d-%m-%Y").date(), axis=1)

In [54]:
(datetime.now().date() - df_abertura["data_abertura"].iloc[0]).days

2030

In [56]:
df_abertura["idade"] = df_abertura.apply(lambda x : (datetime.now().date() - x["data_abertura"]).days/366, axis=1)

In [61]:
df_abertura.sort_values("idade", ascending=False).head()

,cnpj,data_abertura,idade
0,60701190000104,1944-08-22,74.521858
0,07023773000132,1965-12-01,53.289617
0,29912391000100,1966-02-28,53.046448
0,60877859000105,1966-07-01,52.710383
0,60889128000180,1966-07-13,52.677596


In [62]:
df_abertura['idade'].mean()

11.420737477331148

In [63]:
df_abertura.to_excel("../tabelas/idade_empresas.xlsx")

In [74]:
df1 = pd.read_excel("../tabelas/dataset_metricas_score_completo_20190425.xlsx")

In [75]:
df1["cnpj"] = df['cnpj'].astype(str)

In [76]:
df1["cnpj"] = df.apply(lambda x : "0" + x["cnpj"] if len(x["cnpj"])==13 else
                      ("00" + x["cnpj"] if len(x["cnpj"])==12 else x["cnpj"]), axis=1)

In [77]:
df1.head()

,cnpj,segmento,ocorrencias,probabilidade,valor_divida,composicao,fat_medio,pi,lambda,impacto_segmento,risco,dscore,peso,score
2,00272073000302,outros,6,1.0,9659.96,1.0,139381.200000,0.046204,0.046204,0.011551,0.011551,691.914295,1,691.914295
8,05164765000180,outros,2,1.0,3100.56,1.0,94885.300000,0.021785,0.021785,0.005446,0.005446,696.187692,1,696.187692
9,05674948000146,outros,3,1.0,6202.47,1.0,14578.056667,0.283644,0.283644,0.070911,0.070911,650.362280,1,650.362280
11,06135496000197,outros,1,1.0,74.77,1.0,7751.399167,0.006431,0.006431,0.001608,0.001608,698.874633,1,698.874633
12,07127571000130,outros,4,1.0,2648.22,1.0,147721.475000,0.011951,0.011951,0.002988,0.002988,697.908503,1,697.908503


In [78]:
df2 = pd.read_excel("../tabelas/dataset_modelo_201904.xlsx")

In [81]:
df2["cnpj"] = df2["cnpj"].astype(str)

In [83]:
df2["cnpj"] = df2.apply(lambda x : "0" + x["cnpj"] if len(x["cnpj"])==13 else
                        ("00" + x["cnpj"] if len(x["cnpj"])==12 else x["cnpj"]), axis=1)

In [85]:
df2["cnpj"].unique().tolist().__len__()

717

In [87]:
# selecionando apenas os cnpjs eleitos pelos criterios de elegibilidade

In [91]:
df2 = df2[(df2["quantidade_cheques"]==0) & (df2["prop_divida"]<=1.5)]

In [92]:
df2["cnpj"].unique().tolist().__len__()

422

In [94]:
df2.drop(columns=["prop_divida", "quantidade_cheques"], axis=1, inplace=True)

In [101]:
df2 = df2.merge(df_abertura[["cnpj", "idade"]], left_on="cnpj", right_on="cnpj", how="left")

In [102]:
df2.head()

,cnpj,tempo_medio,idade_maxima,credito,infra,outros,processos,dispersao,idade
0,00225356000123,0.000000,1793,0.000000,0.0,1.000000,0.0,0.0,24.508197
1,00265954000207,0.000000,73,0.000000,0.0,1.000000,0.0,0.0,18.773224
2,00272073000302,48.600000,320,0.000000,0.0,1.000000,0.0,0.0,10.852459
3,00437101000124,31.666667,283,0.972151,0.0,0.027849,0.0,0.0,24.144809
4,00661205000118,0.000000,935,1.000000,0.0,0.000000,0.0,0.0,23.795082


In [105]:
df_risco = df1[["cnpj", "risco"]]

In [111]:
df_impacto = df1[["cnpj", "impacto_segmento"]]

In [115]:
df_impacto = df_impacto.groupby("cnpj").mean().reset_index()

In [116]:
df_impacto.shape

(491, 2)

In [117]:
df2.shape

(422, 9)

In [119]:
df2 = df2.merge(df_impacto, left_on='cnpj', right_on='cnpj', how='left')

In [121]:
df2.rename(columns={"impacto_segmento" : 'impacto'}, inplace=True)

In [123]:
df2.head()

,cnpj,tempo_medio,idade_maxima,credito,infra,outros,processos,dispersao,idade,impacto
0,00225356000123,0.000000,1793,0.000000,0.0,1.000000,0.0,0.0,24.508197,0.001192
1,00265954000207,0.000000,73,0.000000,0.0,1.000000,0.0,0.0,18.773224,0.000516
2,00272073000302,48.600000,320,0.000000,0.0,1.000000,0.0,0.0,10.852459,0.011551
3,00437101000124,31.666667,283,0.972151,0.0,0.027849,0.0,0.0,24.144809,0.475850
4,00661205000118,0.000000,935,1.000000,0.0,0.000000,0.0,0.0,23.795082,0.910038


In [129]:
df2.shape

(422, 10)

In [130]:
df2.dropna().shape

(394, 10)

In [132]:
# dominio de email

In [142]:
js = get_details('54cc0469-0e2f-49af-bb2c-8b5580174c60')

In [148]:
js.get("result").get('cnpj').get("email").split("@")[1].split(".")[0]

'hotmail'

In [150]:
from tqdm import tqdm_notebook

In [177]:
resp = []
for el in tqdm_notebook(df['cnpj'].unique().tolist()):
    numero = df[df['cnpj']==el]['numero_consulta'].iloc[0]
    js = get_details(numero)
    email = js.get("result").get("cnpj").get("email")
    if len(email) > 0 and email is not None:
        dom = email.split("@")
        if len(dom) == 1:
            dom = email
        else:
            dom = dom[1].split(".")[0]
    else:
        dom = None
    resp.append(pd.DataFrame({'cnpj' : [el], "dominio" : [dom]}))

TypeError: object of type 'NoneType' has no len()

In [179]:
_df = pd.concat(resp)

In [180]:
_cnpj = _df['cnpj'].unique().tolist()

In [182]:
lista = [el for el in df['cnpj'].unique().tolist() if el not in _cnpj]

In [190]:
for el in tqdm_notebook(lista):
    numero = df[df['cnpj']==el]['numero_consulta'].iloc[0]
    js = get_details(numero)
    email = js.get("result").get("cnpj").get("email")
    if email is None:
        dom = None
    else:
        if len(email) > 0:
            dom = email.split("@")
            if len(dom) == 1:
                dom = "site"
            else:
                dom = dom[1].split(".")[0]
        else:
            dom = None
    resp.append(pd.DataFrame({'cnpj' : [el], "dominio" : [dom]}))

In [191]:
dfdom = pd.concat(resp)

In [192]:
dfdom

,cnpj,dominio
0,19008188000142,festabox
0,00748390000182,festabox
0,00272073000302,wmsassessoria
0,00570256000134,ocilcontabilidade
0,03403405000169,lupecontabilidade
0,04501499000171,exitumempresarial
0,05164765000180,None
0,05674948000146,None
0,05690993000194,patriciabonaldi
0,06135496000197,gigalink


In [193]:
dfdom.groupby("dominio").count().sort_values("cnpj", ascending=False)

,cnpj
dominio,
hotmail,288
gmail,267
uol,93
terra,91
yahoo,72
ig,29
bol,24
outlook,22
globo,11


In [195]:
dfdom[dfdom["dominio"]=='site']

,cnpj,dominio


In [163]:
email.split("@")

['www.ama.org.br']

In [168]:
email = js.get("result").get("cnpj").get('email')

In [170]:
email.split("@")

['www.ama.org.br']